### Ensuring Consistency Across Training & Inference Datasets: Pipeline Integration
**Question**: Create and train a machine learning pipeline that ensures feature transformation consistency across training and inference datasets using scikit-learn.

In [1]:
# write your code from here

import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def load_data():
    try:
        boston = load_boston()
        X = pd.DataFrame(boston.data, columns=boston.feature_names)
        y = pd.Series(boston.target, name='MEDV')
        return X, y
    except Exception as e:
        raise RuntimeError(f"Failed to load Boston Housing dataset: {e}")

def build_pipeline(numerical_features, categorical_features):
    # Preprocessing for numeric data
    numeric_transformer = StandardScaler()

    # Preprocessing for categorical data
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Complete pipeline: preprocessing + model
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42))
    ])
    return pipeline

def main():
    # Load data
    X, y = load_data()

    # Identify categorical and numerical features
    categorical_features = []  # Boston dataset is all numeric, so empty here
    numerical_features = X.columns.tolist()

    # Split train/inference sets
    X_train, X_infer, y_train, y_infer = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Build pipeline
    pipeline = build_pipeline(numerical_features, categorical_features)

    # Fit model
    pipeline.fit(X_train, y_train)

    # Predict on inference set
    y_pred = pipeline.predict(X_infer)

    # Evaluate
    mse = mean_squared_error(y_infer, y_pred)
    print(f"Mean Squared Error on inference set: {mse:.4f}")

    # Demonstrate pipeline consistency with new data
    new_data = X_infer.sample(3, random_state=1)
    print("\nNew data sample (before prediction):")
    print(new_data)

    new_pred = pipeline.predict(new_data)
    print("\nPredictions on new data:")
    print(new_pred)

def test_pipeline_consistency():
    # Simple check that pipeline output shape matches input samples
    X, y = load_data()
    pipeline = build_pipeline(X.columns.tolist(), [])

    X_train, X_infer, y_train, y_infer = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)

    preds_train = pipeline.predict(X_train)
    preds_infer = pipeline.predict(X_infer)

    assert len(preds_train) == len(X_train), "Prediction length mismatch on training set"
    assert len(preds_infer) == len(X_infer), "Prediction length mismatch on inference set"

    print("Pipeline consistency tests passed.")

if __name__ == "__main__":
    main()
    test_pipeline_consistency()

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: